In [1]:
#!pip3 install sympy

In [2]:
# Python libraries to install
# Lesson 1
import time
from datetime import date
from datetime import datetime as dt
from datetime import timedelta

# Lesson 2
import numpy as np
import pandas as pd
import pandas_datareader as dr
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select, WebDriverWait

# Lesson 3 (in addition to above)
from sympy import solve, symbols
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
# Required
company_ticker = "HES"
# or Try:
# 'F'
# 'KHC'
# 'DVN'

# Optional
company_name = "Hess"
# or Try:
# 'Ford Motor'
# 'Kraft Heinz Co'
# 'Devon Energy'

# Optional Input Choices:
# ALL, Annual, Anytime, Bi-Monthly, Monthly, N/A, None, Pays At Maturity, Quarterly, Semi-Annual, Variable
coupon_frequency = "Semi-Annual"

In [4]:
scrape_new_data = True

if scrape_new_data:
    # Selenium script
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()), options=options
    )

    # store starting time
    begin = time.time()

    # FINRA's TRACE Bond Center
    driver.get("http://finra-markets.morningstar.com/BondCenter/Results.jsp")

    # click agree
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, ".button_blue.agree"))
    ).click()

    # click edit search
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "a.qs-ui-btn.blue"))
    ).click()

    # input Issuer Name
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (By.CSS_SELECTOR, "input[id=firscreener-issuer]")
        )
    )
    inputElement = driver.find_element_by_id("firscreener-issuer")
    inputElement.send_keys(company_name)

    # input Symbol
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "input[id=firscreener-cusip]"))
    )
    inputElement = driver.find_element_by_id("firscreener-cusip")
    inputElement.send_keys(company_ticker)

    # click advanced search
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "a.ms-display-switcher.hide"))
    ).click()

    # input Coupon Frequency
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (By.CSS_SELECTOR, "select[name=interestFrequency]")
        )
    )
    Select(
        (driver.find_elements_by_css_selector("select[name=interestFrequency]"))[0]
    ).select_by_visible_text(coupon_frequency)

    # click show results
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "input.button_blue[type=submit]"))
    ).click()

    # wait for results
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (By.CSS_SELECTOR, ".rtq-grid-row.rtq-grid-rzrow .rtq-grid-cell-ctn")
        )
    )

    # create DataFrame from scrape
    frames = []
    for page in range(1, 11):
        bonds = []
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, (f"a.qs-pageutil-btn[value='{str(page)}']"))
            )
        )  # wait for page marker to be on expected page
        time.sleep(2)

        headers = [
            title.text
            for title in driver.find_elements_by_css_selector(
                ".rtq-grid-row.rtq-grid-rzrow .rtq-grid-cell-ctn"
            )[1:]
        ]

        tablerows = driver.find_elements_by_css_selector(
            "div.rtq-grid-bd > div.rtq-grid-row"
        )
        for tablerow in tablerows:
            tablerowdata = tablerow.find_elements_by_css_selector("div.rtq-grid-cell")
            bond = [item.text for item in tablerowdata[1:]]
            bonds.append(bond)

            # Convert to DataFrame
            df = pd.DataFrame(bonds, columns=headers)

        frames.append(df)

        try:
            driver.find_element_by_css_selector("a.qs-pageutil-next").click()
        except:  # noqa E722
            break

    bond_prices_df = pd.concat(frames)

    # store end time
    end = time.time()

    # total time taken
    print(f"Total runtime of the program is {end - begin} seconds")

else:
    bond_prices_df = pd.read_csv("bond-prices.csv")

bond_prices_df

AttributeError: 'WebDriver' object has no attribute 'find_element_by_id'

In [ ]:
def bond_dataframe_filter(df):
    # Drop bonds with missing yields and missing credit ratings
    df["Yield"].replace("", np.nan, inplace=True)
    df["Moody's®"].replace({"WR": np.nan, "": np.nan}, inplace=True)
    df["S&P"].replace({"NR": np.nan, "": np.nan}, inplace=True)
    df = df.dropna(subset=["Yield"])
    df = df.dropna(subset=["Moody's®"])
    df = df.dropna(subset=["S&P"])

    # Create Maturity Years column that aligns with Semi-Annual Payments from corporate bonds
    df["Yield"] = df["Yield"].astype(float)
    df["Coupon"] = df["Coupon"].astype(float)
    df["Price"] = df["Price"].astype(float)
    now = dt.strptime(date.today().strftime("%m/%d/%Y"), "%m/%d/%Y")
    df["Maturity"] = pd.to_datetime(df["Maturity"]).dt.strftime("%m/%d/%Y")
    daystillmaturity = []
    yearstillmaturity = []
    for maturity in df["Maturity"]:
        daystillmaturity.append((dt.strptime(maturity, "%m/%d/%Y") - now).days)
        yearstillmaturity.append((dt.strptime(maturity, "%m/%d/%Y") - now).days / 360)
    df = df.reset_index(drop=True)
    df["Maturity"] = pd.Series(daystillmaturity)
    #         `df['Maturity Years'] = pd.Series(yearstillmaturity).round()` # Better for Annual Payments
    df["Maturity Years"] = (
        round(pd.Series(yearstillmaturity) / 0.5) * 0.5
    )  # Better for Semi-Annual Payments

    # Target bonds with short-term maturities
    df["Maturity"] = df["Maturity"].astype(float)
    # `df = df.loc[df['Maturity'] >= 0]`
    years_mask = (df["Maturity Years"] > 0) & (df["Maturity Years"] <= 5)
    df = df.loc[years_mask]
    return df

In [ ]:
bond_df_result = bond_dataframe_filter(bond_prices_df)
bond_df_result

In [ ]:
# Ten-Year Risk-free Rate
timespan = 100
current_date = date.today()
past_date = current_date - timedelta(days=timespan)
ten_year_risk_free_rate_df = dr.DataReader("^TNX", "yahoo", past_date, current_date)
ten_year_risk_free_rate = (
    ten_year_risk_free_rate_df.iloc[len(ten_year_risk_free_rate_df) - 1, 5]
) / 100
ten_year_risk_free_rate

In [ ]:
# Market Risk Premium
market_risk_premium = 0.0472

In [ ]:
# Market Equity Beta
stock_market_beta = 1

In [ ]:
# Market Rate of Return
market_rate_of_return = ten_year_risk_free_rate + (
    stock_market_beta * market_risk_premium
)
market_rate_of_return

In [ ]:
# One-Year Risk-free Rate
one_year_risk_free_rate = (1 + ten_year_risk_free_rate) ** (1 / 10) - 1
one_year_risk_free_rate

In [ ]:
# Vanguard Short-Term Corporate Bond Index Fund ETF Shares
bond_fund_ticker = "VCSH"

In [ ]:
# Download data for the bond fund and the market
market_data = dr.get_data_yahoo("SPY", past_date, current_date)  # the market
fund_data = dr.get_data_yahoo("VCSH", past_date, current_date)  # the bond fund

In [ ]:
# Approach #1 - Covariance/Variance Method:

# Calculate the covariance between the fund and the market -- this is the numerator in the Beta calculation
fund_market_cov = fund_data["Adj Close"].cov(market_data["Adj Close"])
print("covariance between fund and market: ", fund_market_cov)

# Calculate market (S&P) variance -- this is the denominator in the Beta calculation
market_var = market_data["Adj Close"].var()
print("market variance: ", market_var)

# Calculate Beta
bond_fund_beta_cv = fund_market_cov / market_var
print("bond fund beta (using covariance/variance): ", bond_fund_beta_cv)

In [ ]:
# Approach #2 - Correlation Method:

# Calculate the standard deviation of the market by taking the square root of the variance, for use in the denominator
market_stdev = market_var ** 0.5
print("market standard deviation: ", market_stdev)

# Calculate bond fund standard deviation, for use in the numerator

fund_stdev = fund_data["Adj Close"].std()
print("fund standard deviation: ", fund_stdev)

# Calculate Pearson correlation between bond fund and market (S&P), for use in the numerator
fund_market_Pearson_corr = fund_data["Adj Close"].corr(
    market_data["Adj Close"], method="pearson"
)
print("Pearson correlation between fund and market: ", fund_market_Pearson_corr)

# Calculate Beta
fund_beta_corr = fund_stdev * fund_market_Pearson_corr / market_stdev
print("bond fund beta (using correlation): ", fund_beta_corr)

In [ ]:
# Bond's Beta: use the result of either of the two above approaches, bond_fund_beta_cv or fund_beta_corr
bond_beta = fund_beta_corr
bond_beta

In [ ]:
# Expected Risk Premium
expected_risk_premium = (market_rate_of_return - one_year_risk_free_rate) * bond_beta
expected_risk_premium

In [ ]:
# One-Year Risk-free Rate (same code as above)
one_year_risk_free_rate = (1 + ten_year_risk_free_rate) ** (1 / 10) - 1
one_year_risk_free_rate

In [ ]:
# Risk-adjusted Discount Rate
risk_adjusted_discount_rate = one_year_risk_free_rate + expected_risk_premium
risk_adjusted_discount_rate

In [ ]:
def bonds_probability_of_default(
    coupon, maturity_years, bond_price, principal_payment, risk_adjusted_discount_rate
):

    price = bond_price
    prob_default_exp = 0

    #     `times = np.arange(1, maturity_years+1)` # For Annual Cashflows
    #     annual_coupon = coupon # For Annual Cashflows
    times = np.arange(0.5, (maturity_years - 0.5) + 1, 0.5)  # For Semi-Annual Cashflows
    semi_annual_coupon = coupon / 2  # For Semi-Annual Cashflows

    # Calculation of Expected Cash Flow
    cashflows = np.array([])
    for i in times[:-1]:
        #         cashflows = np.append(cashflows, annual_coupon) # For Annual Cashflows
        #     cashflows = np.append(cashflows, annual_coupon+principal_payment)#  For Annual Cashflows
        cashflows = np.append(
            cashflows, semi_annual_coupon
        )  # For Semi-Annual Cashflows
    cashflows = np.append(
        cashflows, semi_annual_coupon + principal_payment
    )  # For Semi-Annual Cashflows

    for i in range(len(times)):
        #         This code block is used if there is only one payment remaining
        if len(times) == 1:
            prob_default_exp += (
                cashflows[i] * (1 - P) + cashflows[i] * recovery_rate * P
            ) / np.power((1 + risk_adjusted_discount_rate), times[i])
        #         This code block is used if there are multiple payments remaining
        else:
            #             For Annual Cashflows
            #             if times[i] == 1:
            #                 prob_default_exp += ((cashflows[i]*(1-P) + principal_payment*recovery_rate*P) / \
            #                                     np.power((1 + risk_adjusted_discount_rate), times[i]))
            #             For Semi-Annual Cashflows
            if times[i] == 0.5:
                prob_default_exp += (
                    cashflows[i] * (1 - P) + principal_payment * recovery_rate * P
                ) / np.power((1 + risk_adjusted_discount_rate), times[i])
            #             Used for either Annual or Semi-Annual Cashflows
            else:
                prob_default_exp += (
                    np.power((1 - P), times[i - 1])
                    * (cashflows[i] * (1 - P) + principal_payment * recovery_rate * P)
                ) / np.power((1 + risk_adjusted_discount_rate), times[i])

    prob_default_exp = prob_default_exp - price
    implied_prob_default = solve(prob_default_exp, P)
    implied_prob_default = round(float(implied_prob_default[0]) * 100, 2)

    if implied_prob_default < 0:
        return 0.0
    else:
        return implied_prob_default

In [ ]:
# Variables defined for bonds_probability_of_default function
principal_payment = 100
recovery_rate = 0.40
P = symbols("P")

In [ ]:
# This calculation may take some time if there are many coupon payments
bond_df_result.head(1).apply(
    lambda row: bonds_probability_of_default(
        row["Coupon"],
        row["Maturity Years"],
        row["Price"],
        principal_payment,
        risk_adjusted_discount_rate,
    ),
    axis=1,
)

bond_df_result.head(1)